In [1]:
import zipfile
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import radians, cos, sin, asin, sqrt
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path

# Définition du répertoire de travail
HOME_DIR = Path.cwd().parent
DATA_DIR = Path(HOME_DIR, "data")
print(f"Work directory: {HOME_DIR} \nData directory: {DATA_DIR}")

Work directory: /home/onyxia/work/projet-Python/projet 
Data directory: /home/onyxia/work/projet-Python/projet/data


In [3]:
# Chemins
zip_path = Path(DATA_DIR, "BTT_TD_POP1A_2017.CSV.zip")  # Fichier ZIP
extract_path = Path(DATA_DIR)  # Dossier où extraire les fichiers

# Nom du fichier à extraire depuis l'archive
file_to_extract = "BTT_TD_POP1A_2017.CSV"

# Extraction
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extraire uniquement un fichier spécifique
    effectifs = zip_ref.extract(file_to_extract, extract_path)
    print(f"Fichier {file_to_extract} extrait avec succès dans {extract_path} !")


Fichier BTT_TD_POP1A_2017.CSV extrait avec succès dans /home/onyxia/work/projet-Python/projet/data !


In [9]:
Data_pop = pd.read_csv(Path(DATA_DIR,"BTT_TD_POP1A_2017.CSV"),sep=';')
# on affiche les colonnes qui existe dans cette data 
print(Data_pop.columns)

# on affiche encroe les types
print(Data_pop.dtypes)

# et encore les valeurs manquantes
print(Data_pop.isnull().sum())

Index(['NIVGEO', 'CODGEO', 'LIBGEO', 'SEXE', 'AGEPYR10', 'NB'], dtype='object')
NIVGEO       object
CODGEO       object
LIBGEO       object
SEXE          int64
AGEPYR10      int64
NB          float64
dtype: object
NIVGEO       0
CODGEO       0
LIBGEO       0
SEXE         0
AGEPYR10     0
NB          20
dtype: int64


In [10]:
# La variable NB présente des valeurs manquantes. 
# Nous allons donc récupérer les codes postaux pour lesquels cette variable contient une valeur manquante.
missing_nb_rows = Data_pop[Data_pop['NB'].isnull()]
missing_PC = missing_nb_rows['CODGEO'].unique()

print("Codes postaux avec des valeurs manquantes dans la colonne 'NB':")
print(missing_PC)

Codes postaux avec des valeurs manquantes dans la colonne 'NB':
[14666]


In [19]:
# 14666 est hors region parisiene 
# maintenant on va reccuperer les classe d'age present dans la dat set resultat_merge 
merged_data= pd.read_csv(Path(DATA_DIR,'resultat_merge.csv'), sep =',')
print(merged_data['cla_age_5'].unique())

['85-89' '90-94' '95et+' 'tsage' '00-04' '05-09' '10-14' '15-19' '20-24'
 '25-29' '30-34' '35-39' '40-44' '45-49' '50-54' '55-59' '60-64' '65-69'
 '70-74' '75-79' '80-84']


In [20]:
# very nice maintenant on va créer ces classe d'age dans notre Data_pop

# on Remplace les valeurs spéciales "95et+" et "tsage" dans AGEPYR10
Data_pop.loc[Data_pop["AGEPYR10"] == "95et+", "AGEPYR10"] = "95"
Data_pop.loc[Data_pop["AGEPYR10"] == "tsage", "AGEPYR10"] = "999"

# On Convertie la colonne AGEPYR10 en numérique
Data_pop["AGEPYR10"] = pd.to_numeric(Data_pop["AGEPYR10"], errors="coerce")

# oN DéfiniE les bornes (bins) et les labels (classe d’âge)
bins = [
    -1, 4, 9, 14, 19, 24, 29,
    34, 39, 44, 49, 54, 59,
    64, 69, 74, 79, 84, 89,
    94, 999
]
labels = [
    "00-04","05-09","10-14","15-19","20-24","25-29",
    "30-34","35-39","40-44","45-49","50-54","55-59",
    "60-64","65-69","70-74","75-79","80-84","85-89",
    "90-94","95et+"
]

Data_pop["classe_age"] = pd.cut(
    Data_pop["AGEPYR10"],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

In [24]:
# Extraire le département à partir de CODGEO (2 premiers caractères)
Data_pop["dept"] =Data_pop["CODGEO"].astype(str).str[:2]

#  Filtrer sur la région parisienne (IDF)
idf_depts = ["75", "77", "78", "91", "92", "93", "94", "95"]
Data_pop_idf = Data_pop[Data_pop["dept"].isin(idf_depts)].copy()

# ---------------------------------------------------------------------------
# Grouper par dept, SEXE, classe_age, et sommer la population (NB)
Data_pop_final = Data_pop_idf.groupby(["dept", "SEXE", "classe_age"], dropna=True)["NB"].sum().reset_index()


In [25]:
Data_pop_final.head()

,dept,SEXE,classe_age,NB
0,75,1,00-04,127357.393355
1,75,1,05-09,101504.995218
2,75,1,10-14,139088.495826
3,75,1,15-19,206021.110999
4,75,1,20-24,0.000000


Nous remarquons que certaines classes d'âge présentent des valeurs nulles pour NB.
Cela est dû au fait qu'il existe des intervalles d'âge dans resultat_merge qui ne sont pas présents dans Data_pop.

In [26]:
#Comme avant on va reccuperer les classes d'ages et departement  pour les quelles NB = 0 

# On  Filtre les lignes où 'NB' est égal à 0 dans Data_pop_final
nb_zero_rows = Data_pop_final[Data_pop_final['NB'] == 0]

# Puis on  Récupére les classes d'âge et départements correspondants
age_and_dept_with_nb_zero = nb_zero_rows[['classe_age', 'dept']].drop_duplicates()

# Affichage des résultats
print("Classes d'âge et départements pour lesquels NB = 0 :")
print(age_and_dept_with_nb_zero)


Classes d'âge et départements pour lesquels NB = 0 :
    classe_age dept
4        20-24   75
6        30-34   75
7        35-39   75
9        45-49   75
10       50-54   75
..         ...  ...
294      70-74   95
295      75-79   95
297      85-89   95
298      90-94   95
299      95et+   95

[88 rows x 2 columns]


In [27]:
# Calculer le nombre total de lignes par département
total_rows_per_dept = Data_pop_final.groupby('dept').size()

# Calculer le nombre de lignes où 'NB' est égal à 0 par département
nb_zero_per_dept = Data_pop_final[Data_pop_final['NB'] == 0].groupby('dept').size()

# Calculer le pourcentage des 'NB = 0' pour chaque département
percentage_nb_zero_per_dept = (nb_zero_per_dept / total_rows_per_dept * 100).reset_index(name='percentage_NB_zero')

# Renommer les colonnes pour plus de clarté
percentage_nb_zero_per_dept.columns = ['dept', 'percentage_NB_zero']

# Afficher les résultats
print("Pourcentage des NB = 0 dans chaque département :")
print(percentage_nb_zero_per_dept)


Pourcentage des NB = 0 dans chaque département :
  dept  percentage_NB_zero
0   75                55.0
1   77                55.0
2   78                55.0
3   91                55.0
4   92                55.0
5   93                55.0
6   94                55.0
7   95                55.0
